In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import scipy.stats as st
from scipy.stats import linregress
import ipywidgets as widgets
import gmaps

from config import weather_api_key

In [24]:
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [39]:
# 1. Import the WeatherPy_database.csv file. 
challenge_city_data_df = pd.read_csv("Vacation_Search/WeatherPy_Database.csv")
challenge_city_data_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,mar del plata,AR,2/25/2022 4:16,-38.0023,-57.5575,65.08,83,0,7.00,clear sky,0,0
1,1,mataura,NZ,2/25/2022 4:16,-46.1927,168.8643,72.77,46,1,4.65,clear sky,0,0
2,2,rikitea,PF,2/25/2022 4:11,-23.1203,-134.9692,79.86,67,31,6.51,scattered clouds,0,0
3,3,ushuaia,AR,2/25/2022 4:16,-54.8000,-68.3000,49.66,93,75,3.44,broken clouds,0,0
4,4,batagay-alyta,RU,2/25/2022 4:16,67.8006,130.4114,4.78,84,100,11.95,overcast clouds,0,0
5,5,santa maria,BR,2/25/2022 4:16,-29.6842,-53.8069,75.43,62,92,8.48,overcast clouds,0,0
6,6,gafanha da encarnacao,PT,2/25/2022 4:16,40.6181,-8.7330,56.77,51,100,7.18,overcast clouds,0,0
7,7,itoman,JP,2/25/2022 4:16,26.1247,127.6694,64.78,50,75,13.80,broken clouds,0,0
8,8,busselton,AU,2/25/2022 4:16,-33.6500,115.3333,85.05,44,76,7.56,broken clouds,0,0
9,9,nouadhibou,MR,2/25/2022 4:16,20.9310,-17.0347,62.58,88,1,17.27,clear sky,0,0


In [40]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [47]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = challenge_city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (challenge_city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
1,1,mataura,NZ,2/25/2022 4:16,-46.1927,168.8643,72.77,46,1,4.65,clear sky,0,0
3,3,ushuaia,AR,2/25/2022 4:16,-54.8000,-68.3000,49.66,93,75,3.44,broken clouds,0,0
5,5,santa maria,BR,2/25/2022 4:16,-29.6842,-53.8069,75.43,62,92,8.48,overcast clouds,0,0
9,9,nouadhibou,MR,2/25/2022 4:16,20.9310,-17.0347,62.58,88,1,17.27,clear sky,0,0
16,16,atambua,ID,2/25/2022 4:16,-9.1061,124.8925,75.96,88,100,9.28,overcast clouds,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,567,soe,ID,2/25/2022 4:19,-9.8607,124.2840,69.01,90,100,5.03,overcast clouds,0,0
572,572,akureyri,IS,2/25/2022 4:20,65.6835,-18.0878,8.53,78,0,5.75,clear sky,0,0
576,576,kampene,CD,2/25/2022 4:20,-3.6000,26.6667,66.09,94,99,1.95,overcast clouds,0,0
591,591,palauig,PH,2/25/2022 4:20,15.4350,119.9045,84.96,63,4,9.10,clear sky,0,0


In [48]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                     0
City                        0
Country                     2
Date                        0
Lat                         0
Lng                         0
Max Temp                    0
Humidity                    0
Cloudiness                  0
Wind Speed                  0
Current Description         0
Rain inches (last 3 hrs)    0
Snow inches (last 3 hrs)    0
dtype: int64

In [49]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
1,1,mataura,NZ,2/25/2022 4:16,-46.1927,168.8643,72.77,46,1,4.65,clear sky,0,0
3,3,ushuaia,AR,2/25/2022 4:16,-54.8000,-68.3000,49.66,93,75,3.44,broken clouds,0,0
5,5,santa maria,BR,2/25/2022 4:16,-29.6842,-53.8069,75.43,62,92,8.48,overcast clouds,0,0
9,9,nouadhibou,MR,2/25/2022 4:16,20.9310,-17.0347,62.58,88,1,17.27,clear sky,0,0
16,16,atambua,ID,2/25/2022 4:16,-9.1061,124.8925,75.96,88,100,9.28,overcast clouds,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,567,soe,ID,2/25/2022 4:19,-9.8607,124.2840,69.01,90,100,5.03,overcast clouds,0,0
572,572,akureyri,IS,2/25/2022 4:20,65.6835,-18.0878,8.53,78,0,5.75,clear sky,0,0
576,576,kampene,CD,2/25/2022 4:20,-3.6000,26.6667,66.09,94,99,1.95,overcast clouds,0,0
591,591,palauig,PH,2/25/2022 4:20,15.4350,119.9045,84.96,63,4,9.10,clear sky,0,0


In [53]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = " "
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,mataura,NZ,72.77,clear sky,-46.1927,168.8643,
3,ushuaia,AR,49.66,broken clouds,-54.8000,-68.3000,
5,santa maria,BR,75.43,overcast clouds,-29.6842,-53.8069,
9,nouadhibou,MR,62.58,clear sky,20.9310,-17.0347,
16,atambua,ID,75.96,overcast clouds,-9.1061,124.8925,
21,pemberton,CA,26.46,clear sky,50.3164,-122.8027,
29,vaini,TO,86.16,broken clouds,-21.2000,-175.2000,
30,punta arenas,CL,48.31,clear sky,-53.1500,-70.9167,
31,thompson,CA,-2.04,few clouds,55.7435,-97.8558,
34,saldanha,ZA,64.42,clear sky,-33.0117,17.9442,


In [54]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.") 

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [55]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,mataura,NZ,72.77,clear sky,-46.1927,168.8643,Ellie's Villa
3,ushuaia,AR,49.66,broken clouds,-54.8000,-68.3000,Albatross Hotel
5,santa maria,BR,75.43,overcast clouds,-29.6842,-53.8069,Hotel Morotin
9,nouadhibou,MR,62.58,clear sky,20.9310,-17.0347,El Medina
16,atambua,ID,75.96,overcast clouds,-9.1061,124.8925,Pastori 1 Jemaat Polycarpus
...,...,...,...,...,...,...,...
566,ornskoldsvik,SE,8.42,overcast clouds,63.2909,18.7153,ファーストホテル スタット
567,soe,ID,69.01,overcast clouds,-9.8607,124.2840,Dena Hotel
572,akureyri,IS,8.53,clear sky,65.6835,-18.0878,Icelandair Hotel Akureyri
591,palauig,PH,84.96,clear sky,15.4350,119.9045,Alperi Botique Resort


In [56]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [57]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [58]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))